In [1]:
import numpy as np
from asdf import AsdfFile
from astropy.io import fits
from astropy import wcs as astwcs
from gwcs import wcs

from jwst import datamodels
from jwst.assign_wcs import nirspec
from jwst.transforms import models

In [2]:
# These are the CV3 files
# It uses the new ifupost and ote files
refs = {'camera': '/grp/crds/cache/references/jwst/jwst_nirspec_camera_0004.asdf',
        'collimator': '/grp/crds/cache/references/jwst/jwst_nirspec_collimator_0004.asdf',
        'disperser': '/grp/crds/cache/references/jwst/jwst_nirspec_disperser_0034.asdf',
        'distortion': 'N/A',
        'filteroffset': 'N/A',
        'fore': '/grp/crds/cache/references/jwst/jwst_nirspec_fore_0022.asdf',
        'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf',
        'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf',
        'ifupost': 'ifupost.asdf',
        'ifuslicer': '/grp/crds/cache/references/jwst/jwst_nirspec_ifuslicer_0003.asdf',
        'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf',
        'ote': 'ote.asdf',
        'regions': 'N/A',
        'specwcs': 'N/A',
        'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf'}


In [3]:
wcs_kw = {'wcsaxes': 2, 'ra_ref': 165, 'dec_ref': 54,
          'v2_ref': -8.3942412, 'v3_ref': -5.3123744, 'roll_ref': 37,
          'crpix1': 1024, 'crpix2': 1024,
          'cdelt1': .08, 'cdelt2': .08,
          'ctype1': 'RA---TAN', 'ctype2': 'DEC--TAN',
          'pc1_1': 1, 'pc1_2': 0, 'pc2_1': 0, 'pc2_2': 1
          }

slit_fields_num = ["shutter_id", "xcen", "ycen",
                   "ymin", "ymax", "quadrant", "source_id",
                   "stellarity", "source_xpos", "source_ypos"]


slit_fields_str = ["name", "shutter_state", "source_name", "source_alias"]


In [4]:
def create_hdul(detector='NRS1'):
    """
    Create a fits HDUList instance.
    """
    hdul = fits.HDUList()
    phdu = fits.PrimaryHDU()
    phdu.header['instrume'] = 'NIRSPEC'
    phdu.header['detector'] = detector
    phdu.header['time-obs'] = '8:59:37'
    phdu.header['date-obs'] = '2016-09-05'

    scihdu = fits.ImageHDU()
    scihdu.header['EXTNAME'] = "SCI"
    for item in wcs_kw.items():
        scihdu.header[item[0]] = item[1]
    hdul.append(phdu)
    hdul.append(scihdu)
    return hdul

In [5]:
def create_nirspec_ifu_file(filter, grating, lamp='N/A', detector='NRS1'):
    image = create_hdul(detector)
    image[0].header['exp_type'] = 'NRS_IFU'
    image[0].header['filter'] = filter
    image[0].header['grating'] = grating
    image[1].header['crval3'] = 0
    image[1].header['wcsaxes'] = 3
    image[1].header['ctype3'] = 'WAVE'
    image[0].header['lamp'] = lamp
    image[0].header['GWA_XTIL'] = 0.35986012
    image[0].header['GWA_YTIL'] = 0.13448857
    image[0].header['GWA_TILT'] = 37.1
    return image

In [6]:
!ulimit -n 1024
hdul = create_nirspec_ifu_file(grating='PRISM', filter='CLEAR',  detector='NRS1')
im = datamodels.ImageModel(hdul)

pipeline = nirspec.create_pipeline(im, refs)
w = wcs.WCS(pipeline)
im.meta.wcs = w

2018-06-19 14:28:14,173 - stpipe - INFO - gwa_ytilt is 0.13448857 deg
2018-06-19 14:28:14,173 - stpipe - INFO - gwa_xtilt is 0.35986012 deg
2018-06-19 14:28:14,174 - stpipe - INFO - theta_y correction: -0.02747388780339539 deg
2018-06-19 14:28:14,175 - stpipe - INFO - theta_x correction: 0.0 deg
2018-06-19 14:28:27,332 - stpipe - INFO - Created a NIRSPEC nrs_ifu pipeline with references {'ifufore': '/grp/crds/cache/references/jwst/jwst_nirspec_ifufore_0003.asdf', 'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_nirspec_wavelengthrange_0004.asdf', 'fpa': '/grp/crds/cache/references/jwst/jwst_nirspec_fpa_0005.asdf', 'ifuslicer': '/grp/crds/cache/references/jwst/jwst_nirspec_ifuslicer_0003.asdf', 'specwcs': 'N/A', 'camera': '/grp/crds/cache/references/jwst/jwst_nirspec_camera_0004.asdf', 'msa': '/grp/crds/cache/references/jwst/jwst_nirspec_msa_0005.asdf', 'ifupost': 'ifupost.asdf', 'distortion': 'N/A', 'regions': 'N/A', 'ote': 'ote.asdf', 'filteroffset': 'N/A', 'disperser': '/grp/

In [7]:
# Setup the test
slitx = [0] * 5
slity = [-.5, -.25, 0, .25, .5]
wave_range = [6e-07, 5.3e-06]
lam = np.array([.7e-7, 1e-6, 2e-6, 3e-6, 5e-6])

order, wrange = nirspec.get_spectral_order_wrange(im, refs['wavelengthrange'])
im.meta.wcsinfo.sporder = order
im.meta.wcsinfo.waverange_start = wrange[0]
im.meta.wcsinfo.waverange_end = wrange[1]
# Use slice 0
slit_wcs = nirspec.nrs_wcs_set_input(im, 0)


In [8]:
# Slit to MSA entrance
# This includes the Slicer transform and the IFUFORE transform
slit2msa = slit_wcs.get_transform('slit_frame', 'msa_frame')
msax, msay, _= slit2msa(slitx, slity, lam)
print('slitx: ', slitx )
print('slity: ', slity)
print('msax: ', msax)
print('msay: ', msay)


slitx:  [0, 0, 0, 0, 0]
slity:  [-0.5, -0.25, 0, 0.25, 0.5]
msax:  [ 0.04131163  0.04131081  0.04131053  0.04131081  0.04131163]
msay:  [ -6.16115660e-04  -3.08195113e-04   3.43005693e-09   3.08202601e-04
   6.16125124e-04]


In [9]:
# Slicer
slit2slicer = slit_wcs.get_transform('slit_frame', 'slicer')
x_slicer, y_slicer, _ = slit2slicer(slitx, slity, lam)
print('x_slicer: ', x_slicer)
print('y_slicer:', y_slicer)

x_slicer:  [ 0.0004  0.0004  0.0004  0.0004  0.0004]
y_slicer: [-0.006 -0.003  0.     0.003  0.006]


In [10]:
# MSA exit
# Applies the IFUPOST transform to coordinates at the Slicer
ifupost = datamodels.IFUPostModel('ifupost.asdf')
ifupost_transform = nirspec._create_ifupost_transform(ifupost.slice_0)
x_msa_exit, y_msa_exit = ifupost_transform(x_slicer, y_slicer, lam)
ifupost.close()
print('x_msa_exit: ', x_msa_exit)
print('y_msa_exit: ', y_msa_exit)

x_msa_exit:  [ 0.04862196  0.0486194   0.04861821  0.04861838  0.04861992]
y_msa_exit:  [ 0.00794006  0.00824181  0.00854343  0.0088448   0.00914578]


In [11]:
# Coordinates at Collimator exit
# Applies the Collimator forward transform to coordinates at the MSA exit
col = datamodels.open(refs['collimator'])
colx, coly = col.model.inverse(x_msa_exit, y_msa_exit)
col.close()
print('x_collimator_exit', colx)
print('y_collimator_exit', coly)

x_collimator_exit [ 0.07443995  0.074434    0.07443021  0.07442859  0.07442913]
y_collimator_exit [-0.28242038 -0.28192617 -0.28143233 -0.28093905 -0.28044655]


In [12]:
# Slit to GWA entrance
# applies the Collimator forward, Unitless to Directional and 3D Rotation to MSA exit coordinates
disp = datamodels.DisperserModel(refs['disperser'])
disperser = nirspec.correct_tilt(disp, im.meta.instrument.gwa_xtilt, im.meta.instrument.gwa_ytilt)
collimator2gwa = nirspec.collimator_to_gwa(refs, disperser)
x_gwa_in, y_gwa_in, z_gwa_in = collimator2gwa(x_msa_exit, y_msa_exit)
disp.close()
print('x_gwa_entrance:' , x_gwa_in)
print('y_gwa_entrance:' , y_gwa_in)
print('z_gwa_entrance:' , z_gwa_in)

2018-06-19 14:28:30,308 - stpipe - INFO - gwa_ytilt is 0.13448857 deg
2018-06-19 14:28:30,309 - stpipe - INFO - gwa_xtilt is 0.35986012 deg
2018-06-19 14:28:30,310 - stpipe - INFO - theta_y correction: -0.02747388780339539 deg
2018-06-19 14:28:30,311 - stpipe - INFO - theta_x correction: 0.0 deg


x_gwa_entrance: [-0.3243966  -0.3244446  -0.32449057 -0.32453449 -0.32457633]
y_gwa_entrance: [-0.27034551 -0.26990593 -0.26946646 -0.26902728 -0.26858858]
z_gwa_entrance: [ 0.90646575  0.90657955  0.90669383  0.90680852  0.90692358]


In [13]:
# Slit to GWA out
#agreq = nirspec.angle_from_disperser(disp, im)
#print(agreq(lam, x_gwa_in, y_gwa_in, z_gwa_in))
slit2gwa = slit_wcs.get_transform('slit_frame', 'gwa')
x_gwa_out, y_gwa_out, z_gwa_out = slit2gwa(slitx, slity, lam)
print('x_gwa_exit:' , x_gwa_out)
print('y_gwa_exit:' , y_gwa_out)
print('z_gwa_exit:' , z_gwa_out)

x_gwa_exit: [-0.41469069 -0.47249373 -0.46962239 -0.46617283 -0.45529259]
y_gwa_exit: [ 0.27034551  0.26990593  0.26946646  0.26902728  0.26858858]
z_gwa_exit: [ 0.86887568  0.83898776  0.84073934  0.84279726  0.84886326]


In [14]:
# CAMERA entrance (assuming direction is from sky to detector)
angles = [disperser['theta_x'], disperser['theta_y'],
          disperser['theta_z'], disperser['tilt_y']]
rotation = models.Rotation3DToGWA(angles, axes_order="xyzy", name='rotation')
dircos2unitless = models.DirCos2Unitless()

gwa2cam = rotation.inverse | dircos2unitless
x_camera_entrance, y_camera_entrance = gwa2cam(x_gwa_out, y_gwa_out, z_gwa_out)
print('x_camera_entrance:' , x_camera_entrance)
print('y_camera_entrance:' , y_camera_entrance)

x_camera_entrance: [-0.0259538  -0.09381243 -0.09029396 -0.08609438 -0.07308257]
y_camera_entrance: [ 0.28032485  0.28115008  0.28055619  0.27994835  0.27913311]


In [15]:
# at FPA
camera = datamodels.CameraModel(refs['camera'])
x_fpa, y_fpa = camera.model.inverse(x_camera_entrance, y_camera_entrance)
camera.close()
print('x_fpa: ', x_fpa )
print('y_fpa: ' , y_fpa)

x_fpa:  [-0.00792689 -0.02717285 -0.02618148 -0.02499675 -0.02131642]
y_fpa:  [-0.00379232 -0.00388355 -0.00402386 -0.00416465 -0.00431548]


In [16]:
# at SCA
slit2sca = slit_wcs.get_transform('slit_frame', 'sca')
x_sca_nrs1, y_sca_nrs1 = slit2sca(slitx, slity, lam)

# At NRS2
fpa = datamodels.FPAModel(refs['fpa'])
x_sca_nrs2, y_sca_nrs2 = fpa.nrs2_model.inverse(x_fpa, y_fpa)
fpa.close()
print('x_sca1: ', x_sca_nrs1)
print('y_sca1: ' , y_sca_nrs1)
print('x_sca2: ', x_sca_nrs2 )
print('y_sca2: ' , y_sca_nrs2)

x_sca1:  [ 1680.21927388   610.99911049   666.07536121   731.89380074   936.35646729]
y_sca1:  [ 812.81553332  807.74719871  799.95200182  792.13037934  783.7509911 ]
x_sca2:  [ 2561.00329019  3630.22352916  3575.14739488  3509.32907218  3304.8665308 ]
y_sca2:  [ 1234.10985934  1239.16222412  1246.95824362  1254.78084916  1263.16329125]


In [17]:
# at oteip
# Goes through slicer, ifufore, and fore transforms
slit2oteip = slit_wcs.get_transform('slit_frame', 'oteip')
x_oteip, y_oteip, _ = slit2oteip(slitx, slity, lam)
print('x_oteip: ', x_oteip)
print('y_oteip: ' , y_oteip)

x_oteip:  [ 0.05009413  0.05043085  0.05076882  0.05110744  0.05145047]
y_oteip:  [-0.04490203 -0.04451562 -0.04412939 -0.04374378 -0.04335772]


In [18]:
# at v2, v3 [in arcsec]
slit2v23 = slit_wcs.get_transform('slit_frame', 'v2v3')
v2, v3, _ = slit2v23(slitx, slity, lam)
v2 /= 3600
v3 /= 3600
print('v2: ', v2)
print('v3: ' , v3)

v2:  [ 0.08365822  0.08351106  0.08336336  0.08321538  0.08306549]
v3:  [-0.13873477 -0.13856653 -0.13839837 -0.13823048 -0.1380624 ]


In [19]:
# Save results to an asdf file

fa = AsdfFile()
fa.tree['slitx'] = list(slitx)
fa.tree['slity'] = list(slity)
fa.tree['lam'] = list(lam)
fa.tree['msax'] = list(msax)
fa.tree['msay'] = list(msay)
fa.tree['x_collimator_exit'] = list(colx)
fa.tree['y_collimator_exit'] = list(coly)
fa.tree['x_gwa_entrance'] = list(x_gwa_in)
fa.tree['y_gwa_entrance'] = list(y_gwa_in)
fa.tree['z_gwa_entrance'] = list(z_gwa_in)
fa.tree['x_gwa_exit'] = list(x_gwa_out)
fa.tree['y_gwa_exit'] = list(y_gwa_out)
fa.tree['z_gwa_exit'] = list(z_gwa_out)
fa.tree['x_camera_entrance'] = list(x_camera_entrance)
fa.tree['y_camera_entrance'] = list(y_camera_entrance)
fa.tree['x_fpa'] = list(x_fpa)
fa.tree['y_fpa'] = list(y_fpa)
fa.tree['x_sca_nrs1'] = list(x_sca_nrs1)
fa.tree['y_sca_nrs1'] = list(y_sca_nrs1)
fa.tree['x_sca_nrs2'] = list(x_sca_nrs2)
fa.tree['y_sca_nrs2'] = list(y_sca_nrs2)
fa.tree['x_oteip'] = list(x_oteip)
fa.tree['y_oteip'] = list(y_oteip)
fa.tree['v2'] = list(v2)
fa.tree['v3'] = list(v3)
fa.tree['x_slicer'] = list(x_slicer)
fa.tree['y_slicer'] = list(y_slicer)
fa.tree['x_msa_exit'] = list(x_msa_exit)
fa.tree['y_msa_exit'] = list(y_msa_exit)
fa.write_to("ifu_prism_functional.asdf", all_array_storage="internal")